In [1]:
#IMPORTING LIBRARIE
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark import SparkConf
from pyspark.sql.functions import to_timestamp



import warnings
warnings.filterwarnings('ignore')

# DATA FRAME ORIGINAL - Project Tweets - Pandas

In [2]:
df = pd.read_csv('ProjectTweets.csv')
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


# EDA

In [3]:
df.shape

(1599999, 6)

In [4]:
df.dtypes

0                                                                                                                       int64
1467810369                                                                                                              int64
Mon Apr 06 22:19:45 PDT 2009                                                                                           object
NO_QUERY                                                                                                               object
_TheSpecialOne_                                                                                                        object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    object
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [6]:
df.isnull().sum()

0                                                                                                                      0
1467810369                                                                                                             0
Mon Apr 06 22:19:45 PDT 2009                                                                                           0
NO_QUERY                                                                                                               0
_TheSpecialOne_                                                                                                        0
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    0
dtype: int64

In [7]:
df.loc[-1] = df.columns
df.index = df.index + 1
df.sort_index(inplace=True)

In [8]:
df.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
new_columns = ["Index", "ID", "Date/Time", "Info", "User", "Tweet"]
df.columns = new_columns
df1 = df

In [10]:
df1.head(3)

,Index,ID,Date/Time,Info,User,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [11]:
df1.shape

(1600000, 6)

In [12]:
keywords = ['H1N1', 'Swine flu', 'Influenza A', 'Pandemic flu', 'Flu outbreak', 
            'H1N1 symptoms', 'H1N1 transmission', 'H1N1 vaccine', 'Antiviral medications', 
            'International health emergency', 'Swine Origin Influenza', 'Influenza Vaccine', 
            'Influenza Transmission', 'Influenza Treatment', 'Flu Virus']

In [13]:
filter = df1["Tweet"].str.contains('|'.join(keywords), case=False)
df2 = tweets_fil = df1[filter]

In [14]:
df2.head(5)

,Index,ID,Date/Time,Info,User,Tweet
41612,41612,1675708738,Fri May 01 20:08:51 PDT 2009,NO_QUERY,junoaggy,swine flu attacks HK so i can't go screw you ...
41641,41641,1675712950,Fri May 01 20:09:26 PDT 2009,NO_QUERY,JasJazzy,Swine Flu Why oh why did you come to canada????
41642,41642,1675713013,Fri May 01 20:09:27 PDT 2009,NO_QUERY,fireflyjane,"is congested, feverish and aches...hope I didn..."
41878,41878,1675828003,Fri May 01 20:25:32 PDT 2009,NO_QUERY,rach107,swine flu at jwu hiding outt
42072,42072,1675929811,Fri May 01 20:40:13 PDT 2009,NO_QUERY,lonewolfmedia,They cancelled the First Monday flea market he...


In [15]:
df2.shape

(1402, 6)

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1402 entries, 41612 to 1599128
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Index      1402 non-null   object
 1   ID         1402 non-null   object
 2   Date/Time  1402 non-null   object
 3   Info       1402 non-null   object
 4   User       1402 non-null   object
 5   Tweet      1402 non-null   object
dtypes: object(6)
memory usage: 76.7+ KB


In [17]:
df2.to_parquet("/home/hduser/Desktop/TWEETS_H1N1.parquet")

# Tratamento de dados (FIM)

In [18]:
spark_conf = SparkConf().setMaster("local[*]").setAppName("Tweets_Hadoop")

spark = SparkSession.builder.config(conf=spark_conf).config('spark.sql.session.timeZone', 'UTC').getOrCreate()

sc = spark.sparkContext

sc.setLogLevel('ERROR')



23/12/31 00:19:47 WARN Utils: Your hostname, BDS-2023 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/12/31 00:19:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 00:19:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/31 00:19:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
tweets_spark = spark.read.parquet("hdfs://localhost:9000/CA4/TWETTS_H1N1.parquet")


In [20]:
tweets_spark.printSchema()

root
 |-- Index: long (nullable = true)
 |-- ID: long (nullable = true)
 |-- Date/Time: string (nullable = true)
 |-- Info: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [21]:
tweets_spark.summary().show()

+-------+-----------------+--------------------+--------------------+--------+-----------+--------------------+-----------------+
|summary|            Index|                  ID|           Date/Time|    Info|       User|               Tweet|__index_level_0__|
+-------+-----------------+--------------------+--------------------+--------+-----------+--------------------+-----------------+
|  count|             1402|                1402|                1402|    1402|       1402|                1402|             1402|
|   mean|578804.5085592011|2.0001463479579172E9|                null|    null|       null|                null|578804.5085592011|
| stddev|401653.7588381154|2.1242077954807264E8|                null|    null|       null|                null|401653.7588381154|
|    min|            41612|          1675708738|Fri Jun 05 08:44:...|NO_QUERY|  13zneralc| I feel a cold co...|            41612|
|    25%|           254826|          1824446054|                null|    null|       null|

In [22]:
conf = SparkConf().set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark = spark.builder.config(conf=conf).getOrCreate()

In [23]:
tweets_spark = tweets_spark.withColumn("Date/Time", to_timestamp(tweets_spark["Date/Time"], "EEE MMM dd HH:mm:ss zzz yyyy"))

In [24]:
tweets_spark = tweets_spark.select("ID", "Date/Time", "User", "Tweet")

In [25]:
tweets_spark.show(truncate=True, n=5)

+----------+-------------------+-------------+--------------------+
|        ID|          Date/Time|         User|               Tweet|
+----------+-------------------+-------------+--------------------+
|1675708738|2009-05-02 03:08:51|     junoaggy|swine flu attacks...|
|1675712950|2009-05-02 03:09:26|     JasJazzy|Swine Flu  Why oh...|
|1675713013|2009-05-02 03:09:27|  fireflyjane|is congested, fev...|
|1675828003|2009-05-02 03:25:32|      rach107|swine flu at jwu ...|
|1675929811|2009-05-02 03:40:13|lonewolfmedia|They cancelled th...|
+----------+-------------------+-------------+--------------------+
only showing top 5 rows



In [26]:
tweets_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Date/Time: timestamp (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [27]:
tweets_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Date/Time: timestamp (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



In [28]:
from pyspark.sql.functions import year, month, count, format_string, col

In [29]:
# Verifique a distribuição de tweets por mês
df = tweets_spark.groupBy(year("Date/Time").alias("year"), month("Date/Time").alias("month")).count() \
                 .orderBy(["year", "month"])

# Obtenha a porcentagem
df = df.withColumn("percentage", format_string("%.2f%%", ((col("count")/tweets_spark.count())*100)))

# Mostre o DataFrame resultante
df.show(truncate=False)

+----+-----+-----+----------+
|year|month|count|percentage|
+----+-----+-----+----------+
|2009|5    |569  |40.58%    |
|2009|6    |833  |59.42%    |
+----+-----+-----+----------+



In [30]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover


In [31]:
#read csv file into dataFrame with automatically inferred schema
tweets_csv = spark.read.csv("hdfs://localhost:9000/CA4/tweets.csv", inferSchema=True, header=True)
tweets_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



In [32]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("SentimentText", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



In [33]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1405 ; Testing data rows: 527


In [34]:
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+-------------------------+-----+------------------------------+
|SentimentText            |label|SentimentWords                |
+-------------------------+-----+------------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |
|I adore cheese #thumbs-up|1    |[i, adore, cheese, #thumbs-up]|
+-------------------------+-----+------------------------------+
only showing top 5 rows



In [35]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+-------------------------+-----+------------------------------+---------------------------+
|SentimentText            |label|SentimentWords                |MeaningfulWords            |
+-------------------------+-----+------------------------------+---------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |[adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|[adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |[adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |[adore, cheese, #loveit]   |
|I adore cheese #thumbs-up|1    |[i, adore, cheese, #thumbs-up]|[adore, cheese, #thumbs-up]|
+-------------------------+-----+------------------------------+---------------------------+
only showing top 5 rows



In [36]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+---------------------------+-------------------------------------------+
|label|MeaningfulWords            |features                                   |
+-----+---------------------------+-------------------------------------------+
|1    |[adore, cheese, #bestever] |(262144,[1689,91011,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #brilliant]|(262144,[1689,45361,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #favorite] |(262144,[1689,100089,108624],[1.0,1.0,1.0])|
+-----+---------------------------+-------------------------------------------+
only showing top 3 rows



In [37]:
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


In [38]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+-----+-------------------------------------+-------------------------------------------------------+
|Label|MeaningfulWords                      |features                                               |
+-----+-------------------------------------+-------------------------------------------------------+
|1    |[adore, classical, music, #thumbs-up]|(262144,[88825,100089,102383,131250],[1.0,1.0,1.0,1.0])|
|1    |[adore, coffee, #loveit]             |(262144,[100089,159212,254974],[1.0,1.0,1.0])          |
+-----+-------------------------------------+-------------------------------------------------------+
only showing top 2 rows



In [39]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+-------------------------------------+----------+-----+
|MeaningfulWords                      |prediction|Label|
+-------------------------------------+----------+-----+
|[adore, classical, music, #thumbs-up]|1.0       |1    |
|[adore, coffee, #loveit]             |1.0       |1    |
|[adore, jam, #loveit]                |1.0       |1    |
|[adore, jam, #toptastic]             |1.0       |1    |
+-------------------------------------+----------+-----+
only showing top 4 rows

correct prediction: 520 , total data: 527 , accuracy: 0.9867172675521821


In [40]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace

In [41]:
# Definição de expressões regulares
at_regex = r"@\w+"  # Remove usernames
link_regex = r"http\S+"  # Remove links
rt_regex = r'\bRT\b'  # Remove 'RT'
ss_regex = r'[^\w\s]'  # Remove Special strings
ds_regex = r'\s+'  # Remove spaces

In [42]:
tweets_spark = tweets_spark.withColumn("clean_tweet", regexp_replace("Tweet", at_regex, ""))
tweets_spark = tweets_spark.withColumn("clean_tweet", regexp_replace("clean_tweet", link_regex, ""))
tweets_spark = tweets_spark.withColumn("clean_tweet", regexp_replace("clean_tweet", rt_regex, ""))
tweets_spark = tweets_spark.withColumn("clean_tweet", regexp_replace("clean_tweet", ss_regex, ""))
tweets_spark = tweets_spark.withColumn("clean_tweet", regexp_replace("clean_tweet", ds_regex, " "))

# Exibição dos resultados
tweets_spark.show(truncate=False, n=5)

+----------+-------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|ID        |Date/Time          |User         |Tweet                                                                                                                                          |clean_tweet                                                                                                                     |
+----------+-------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------+
|1675708738|2009-05-02 03:08:51|junoaggy

In [43]:
tokenizer = Tokenizer(inputCol="clean_tweet", outputCol="words")
tokenizedData = tokenizer.transform(tweets_spark)

swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemoved = swr.transform(tokenizedData)

hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericData = hashTF.transform(SwRemoved).select('MeaningfulWords', 'features')


numericData.show(n=3)

+--------------------+--------------------+
|     MeaningfulWords|            features|
+--------------------+--------------------+
|[swine, flu, atta...|(262144,[40873,61...|
|[swine, flu, oh, ...|(262144,[18184,16...|
|[congested, fever...|(262144,[46168,71...|
+--------------------+--------------------+
only showing top 3 rows



In [44]:
prediction = model.transform(numericData)

predictionFinal = prediction.select(
    "MeaningfulWords", "prediction")

In [45]:
predictionFinal.show(truncate = False, n=10)

+----------------------------------------------------------------------------------------------------------------+----------+
|MeaningfulWords                                                                                                 |prediction|
+----------------------------------------------------------------------------------------------------------------+----------+
|[swine, flu, attacks, hk, cant, go, screw, swines]                                                              |0.0       |
|[swine, flu, oh, come, canada]                                                                                  |0.0       |
|[congested, feverish, acheshope, didnt, catch, swine, flu]                                                      |0.0       |
|[swine, flu, jwu, hiding, outt]                                                                                 |0.0       |
|[cancelled, first, monday, flea, market, live, due, quotswine, fluquot, gonna, go, insanity, insanity, insanity]|0.0 

In [46]:
predictionFinal.count()

1402

In [47]:
# Create a column with id following the data's order 
tweets_spark = tweets_spark.withColumn("row_id", monotonically_increasing_id())
predictionFinal = predictionFinal.withColumn("row_id", monotonically_increasing_id())

# join by "row_id"
tweets_pred = tweets_spark.select('row_id','Date/Time','User', 'tweet', 'clean_tweet') \
                    .join(predictionFinal.select('row_id', 'prediction'), "row_id", "inner")
                

# drop column 
tweets_pred = tweets_pred.drop("row_id")

tweets_pred.show()

+-------------------+---------------+--------------------+--------------------+----------+
|          Date/Time|           User|               tweet|         clean_tweet|prediction|
+-------------------+---------------+--------------------+--------------------+----------+
|2009-05-02 03:08:51|       junoaggy|swine flu attacks...|swine flu attacks...|       0.0|
|2009-05-02 03:09:26|       JasJazzy|Swine Flu  Why oh...|Swine Flu Why oh ...|       0.0|
|2009-05-02 03:09:27|    fireflyjane|is congested, fev...|is congested feve...|       0.0|
|2009-05-02 03:25:32|        rach107|swine flu at jwu ...|swine flu at jwu ...|       0.0|
|2009-05-02 03:40:13|  lonewolfmedia|They cancelled th...|They cancelled th...|       0.0|
|2009-05-02 03:45:31|JasperDracoLuvr|is sick-ish...not...|is sickishnot swi...|       1.0|
|2009-05-02 03:49:48|   LauraRepetti|@ddlovato Yay for...| Yay for elbow hi...|       0.0|
|2009-05-02 04:14:17|       pennz0il|I'm getting prett...|Im getting pretty...|       0.0|

In [48]:
!pip install textblob


Defaulting to user installation because normal site-packages is not writeable


In [49]:
!pip install nltk


Defaulting to user installation because normal site-packages is not writeable


In [50]:
from textblob import TextBlob

In [51]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hduser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [52]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Função de análise de sentimento com TextBlob
@udf(FloatType())
def sentiment(tweet):
    return TextBlob(tweet).sentiment.polarity

# Função de análise de sentimento com VADER
@udf(FloatType())
def sentiment_vader(tweet):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(tweet)['compound']

# Aplicar diretamente no DataFrame
tweets_pred = tweets_pred.withColumn("textblob", sentiment(tweets_pred["clean_tweet"])) \
                         .withColumn("vader", sentiment_vader(tweets_pred["clean_tweet"]))


In [53]:
tweets_pred = tweets_pred.withColumn("score", ((col("prediction") + (col("textblob")*1.5) + (col("vader")*1.5)) / 4))

In [54]:
tweets_pred.select("clean_tweet", "prediction", "textblob", "vader", "score").show(n=10)

+--------------------+----------+------------+-------+--------------------+
|         clean_tweet|prediction|    textblob|  vader|               score|
+--------------------+----------+------------+-------+--------------------+
|swine flu attacks...|       0.0|         0.0|-0.6047|-0.22676251083612442|
|Swine Flu Why oh ...|       0.0|         0.0|-0.3818|-0.14317499846220016|
|is congested feve...|       0.0|        -0.1| -0.516| -0.2309999903663993|
|swine flu at jwu ...|       0.0|         0.0|-0.5859| -0.2197125032544136|
|They cancelled th...|       0.0|  0.08712121| -0.946|-0.32207953836768866|
|is sickishnot swi...|       1.0|5.551115E-17| -0.331| 0.12587499991059303|
| Yay for elbow hi...|       0.0|     -0.1875| -0.204|-0.14681249856948853|
|Im getting pretty...|       0.0|      -0.075|-0.3182|-0.14744999818503857|
| Aw poor Nikki Im...|       0.0|        0.08| 0.7488| 0.31079999171197414|
|I just heard on t...|       0.0|        -1.0|-0.7096| -0.6410999894142151|
+-----------

In [55]:
tweets_pred.printSchema()

root
 |-- Date/Time: timestamp (nullable = true)
 |-- User: string (nullable = true)
 |-- tweet: string (nullable = true)
 |-- clean_tweet: string (nullable = true)
 |-- prediction: double (nullable = false)
 |-- textblob: float (nullable = true)
 |-- vader: float (nullable = true)
 |-- score: double (nullable = true)



In [56]:
from pyspark.sql import SparkSession
from hdfs import InsecureClient
from hdfs.util import HdfsError

In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth
from hdfs import InsecureClient
from hdfs.util import HdfsError

def spark_hadoop(df, folder, partitionBy=None, spark=None):
    hdfs_base_path = "hdfs://localhost:9000"
    hdfs_folder_path = f"{hdfs_base_path}/CA4/{folder}"

    client = InsecureClient('http://localhost:9870', user='hduser')

    try:
        # Tenta acessar o caminho diretamente
        client.content(hdfs_folder_path)

        print('Os arquivos já estão no Hadoop. Lendo os arquivos.')
        df = spark.read.parquet(hdfs_folder_path)
    except HdfsError:
        print('Colocando no Hadoop.')
        if partitionBy:
            # Extraindo ano, mês e dia da coluna Date/Time
            df = df.withColumn("year", year("Date/Time"))
            df = df.withColumn("month", month("Date/Time"))
            df = df.withColumn("day", dayofmonth("Date/Time"))
            
            # Incorporando a lógica de particionamento
            df.write.partitionBy("year", "month", "day").parquet(hdfs_folder_path)
            print(f"Salvo em {hdfs_folder_path} particionado por {partitionBy}")
        else:
            df.write.parquet(hdfs_folder_path)
            print(f"Salvo em {hdfs_folder_path}")

    return df


# Chamando a função spark_hadoop com a lógica de particionamento incorporada
df_salvo = spark_hadoop(tweets_pred, folder="sentiment", partitionBy="Date/Time", spark=spark)


Colocando no Hadoop.


Salvo em hdfs://localhost:9000/CA4/sentiment particionado por Date/Time


In [65]:
df_salvo.show(2)

+-------------------+--------+--------------------+--------------------+----------+--------+-------+--------------------+----+-----+---+
|          Date/Time|    User|               tweet|         clean_tweet|prediction|textblob|  vader|               score|year|month|day|
+-------------------+--------+--------------------+--------------------+----------+--------+-------+--------------------+----+-----+---+
|2009-05-02 03:08:51|junoaggy|swine flu attacks...|swine flu attacks...|       0.0|     0.0|-0.6047|-0.22676251083612442|2009|    5|  2|
|2009-05-02 03:09:26|JasJazzy|Swine Flu  Why oh...|Swine Flu Why oh ...|       0.0|     0.0|-0.3818|-0.14317499846220016|2009|    5|  2|
+-------------------+--------+--------------------+--------------------+----------+--------+-------+--------------------+----+-----+---+
only showing top 2 rows

